# UNO-CG performance evaluation: mechanical problem in 3D with mixed BC

### Imports:

In [ ]:
import os
import torch
from torch.utils.data import DataLoader
from unocg.utils.data import MicrostructureDataset
from unocg.problems.mechanical import MechanicalProblem
from unocg.materials.mechanical import LinearElasticity
from unocg.problems import BC
from unocg.solvers.torch import CgSolver
from unocg.preconditioners.torch import UnoPreconditioner, JacobiPreconditioner
from unocg.transforms.fourier import DiscreteMixedTransform
from unocg.utils.plotting import *
from unocg.utils.evaluation import *
from unocg import config
from matplotlib.transforms import Bbox
import time
import h5py

try:
    import pyvista as pv
    pv.set_jupyter_backend("trame")
    pv.global_theme.trame.server_proxy_enabled = True
    use_pyvista = True
except:
    use_pyvista = False
    warnings.warn("pyvista not found")

### Configuration:

In [ ]:
show_plots = True
dtype = torch.float64
torch.set_float32_matmul_precision("high")
torch.set_default_dtype(dtype)
quad_degree = 2
bc = BC.DIRICHLET_TB
device = "cuda" if torch.cuda.is_available() else "cpu"
args = {'device': device, 'dtype': dtype}

### Create problem

In [ ]:
shape = (192, 192, 192)
material = LinearElasticity(n_dim=3, **args)
problem = MechanicalProblem(shape, material=material, bc=bc, quad_degree=quad_degree)

E = torch.tensor([75.0, 400.0], **args)
nu = torch.tensor([0.3, 0.2], **args)
lame_lambda = E * nu / ((1.0 + nu) * (1.0 - 2. * nu))
lame_mu = E / (2.0 * (1.0 + nu))
params = torch.stack([lame_lambda, lame_mu]).T
loadings = 0.05 * torch.tensor([[0.5, -1.0, 0.5, 0.0, 0.0, 0.0]], **args)

### Generate artificial microstructure or load microstructure from file:

In [ ]:
microstructures = MicrostructureDataset(
    file_name=os.path.join(config.data_path, "3d_microstructures.h5"),
    group_name="test",
    **args
)
data_loader = DataLoader(microstructures, batch_size=1, shuffle=False)
microstructure = microstructures[1]
param_fields = problem.get_param_fields(microstructure.unsqueeze(0), params)

if show_plots and use_pyvista:
    pl = pv.Plotter()
    pl.add_volume(microstructure.cpu().float().numpy(), cmap=["red"], shade=True, scalar_bar_args={"vertical": True, "height": 0.9, "n_labels": 2, "title": ""})
    pl.screenshot(filename=os.path.join(config.results_path, "3d_microstructure_2.png"), transparent_background=True, window_size=[600,600])
    pl.show()

### Compute reference solution

Computing the reference solution may take some time depending on the hardware used

In [ ]:
solver = CgSolver(problem)
print("Computing reference solution using unpreconditioned CG...")
solver_start = time.time()
result_ref = solver.solve(param_fields, loadings, rtol=1e-12)
solver_time = time.time() - solver_start
field_ref = problem.compute_field(result_ref["sol"], param_fields, loadings)
print(f"CG solver converged after {result_ref["n_iter"]} iterations and {solver_time:.4f} s")

In [ ]:
disp = field_ref[0, ..., :3, :, :, :]
stress = field_ref[0, ..., 3:, :, :, :]
stress_norm = stress.norm(dim=problem.ch_dim).cpu().detach()

if show_plots and use_pyvista:
    plot_deformed_rve_3d(problem, disp, stress_norm, loadings, deformation_scaling=8,
                         vmin=0, vmax=26, file=os.path.join(config.results_path, "mechanical_3d_mixed_deformed.png"), figsize=[600, 629])

### Load learned UNO preconditioner

In [ ]:
weights_uno = torch.load(os.path.join(config.data_path, "weights_uno_mechanical_3d_mixed.pt"), weights_only=True, map_location=device)
trafo = DiscreteMixedTransform(dim_fourier=[-3, -1], dim_sine=[-2])
uno_prec = UnoPreconditioner(problem, trafo, weights_uno)
unocg_solver = CgSolver(problem, uno_prec)

### Run CG solver with different preconditioners:

The error measures against the reference solution are evaluated in each iteration using `loss_callback`

In [ ]:
rtol = 1e-11
max_iter = 2500
solver_args = {'rtol': rtol, 'max_iter': max_iter, 'verbose': False}
cg_solver = CgSolver(problem)
disp_loss = problem.get_disp_loss(reduction="none")

def loss_callback(sol):
    with torch.inference_mode():
        return {"disp": disp_loss(sol, result_ref["sol"]).unsqueeze(0)}


unocg_result = unocg_solver.solve(param_fields, loadings, loss_callback=loss_callback, **solver_args)
print(f"UNO-CG converged after {unocg_result['n_iter']} iterations")

cg_result = cg_solver.solve(param_fields, loadings, loss_callback=loss_callback, **solver_args)
print(f"CG converged after {cg_result['n_iter']} iterations")

In [ ]:
results = [None, unocg_result, None, None, cg_result]
labels = [None, "UNO-CG", None, None, "CG"]
metrics = ["disp"]
metric_labels = [r"nRMSE $\boldsymbol{\tilde{u}} \, [-]$",]

if show_plots:
    fig, ax = plt.subplots(1, 1, figsize=[plot_width * 0.5, 2.5], dpi=300, squeeze=False)
    plot_convergence(ax, results, labels, colors, metrics, metric_labels, load_names=["x"], rates=None,
                     xmin=0, xmax=max_iter, bounds=False, ymin=1e-10, ymax=1e0, zoom=True, zoom_it=40, zoom_tol=1e-6)
    ax[0,0].legend()
    fig.tight_layout()
    bbox = fig.get_tightbbox()
    bbox = Bbox([[bbox.x0 - 0.05, bbox.y0 - 0.01], [bbox.x1 + 0.0, bbox.y1 + 0.05]])
    plt.savefig(os.path.join(config.results_path, "convergence_mechanical_3d_mixed.pdf"), dpi=300, bbox_inches=bbox)
    plt.savefig(os.path.join(config.results_path, "convergence_mechanical_3d_mixed.png"), dpi=300, bbox_inches=bbox)
    plt.show()

### Runtime measurements

In [ ]:
cg_module = cg_solver.get_module(**args, rtol=1e-6)
unocg_module = unocg_solver.get_module(**args, rtol=1e-6)

In [ ]:
benchmark_cg(unocg_module, param_fields, loadings, device=device, n_runs=10)

In [ ]:
benchmark_cg(cg_module, param_fields.to(**args), loadings.to(**args), device=device, n_runs=10)